# **Final Presentation**



In this notebook team Power_Factor will describe its developed package: the different functionalities, the completed assignments that led to the developement of the package as well as some code demonstrations and collaboration discussion

In [4]:
from typing import List, Tuple

import networkx as nx
import json
import pprint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse import csr_array
from IPython.display import display
from scipy.sparse.csgraph import connected_components
from power_grid_model.utils import json_deserialize, json_serialize
from power_grid_model.validation import ValidationException, assert_valid_batch_data, assert_valid_input_data
from power_grid_model import (
    CalculationMethod,
    CalculationType,
    ComponentType,
    DatasetType,
    PowerGridModel,
    initialize_array,
)

# **Assignment 1 -graph processing**

In assignment 1 the task is to build a graph processing class. For a given undirected graph as input, there are two functionalities to implement:
- Find downstream vertices - given a specific edge from the graph, find the downstream vertices of the edge, including the downstream vertex of the edge itself
- Find alternative edges - given a specific enabled edge from the graph, returns a list that indicates which currently disabled edges can be enabled so that the graph is again fully connected and acyclic
            

# **Check input validity**

Before the either of the two functionalities are used, the input data must be checked for validity. There are 7 criteria that are evaluated:
1. The vertex (node) and edge ids should be unique
2. The number of edges (or connections between two separate vertices) should have the same as the number of edge ids
3. The vertices connected by the specified edges should be valid vertex ids
4. The number of enabled/disabled edges should be the same as the number of edge ids
5. The id of the source vertex should be a valid vertex id
6. The graph should be fully connected
7. The graph should not contain cycles

Furthermore, Find alternative edges functionality it is important to check if the edge to be disabled has a valid id and whether it is already disabled.

In [7]:
class IDNotFoundError(Exception):
    "Raised when a source or edge id is not found/valid"


class InputLengthDoesNotMatchError(Exception):
    "Raised when number of enabled and disabled edges does not match number of total edges or number of vertex pairs does not match number of edges"


class IDNotUniqueError(Exception):
    "Raised when vertex or edge ids are not unique"


class GraphNotFullyConnectedError(Exception):
    "Raised when graph contains more than 1 component"


class GraphCycleError(Exception):
    "Raised when graph contains a cycle"


class EdgeAlreadyDisabledError(Exception):
    "Edge is already disabled"


def check_unique(vertex_ids, edge_ids): #checks vertex ids or edge ids are unique
    ver = list(set(vertex_ids)) #a set is used because sets contain only unique values
    edge = list(set(edge_ids))
    if len(ver) != len(vertex_ids) or len(edge) != len(edge_ids):
        raise IDNotUniqueError("Vertex or edge ids are not unique")


def check_length_pairs(edge_vertex_id_pairs, edge_ids): #checks if number of vertex pairs matches the number of edges
    if len(edge_vertex_id_pairs) != len(edge_ids):
        raise InputLengthDoesNotMatchError("Number of vertex pairs does not match number of edges")


def check_found_pairs(edge_vertex_id_pairs, vertex_ids): #checks if all vertex pairs contain valid vertex ids
    if all(all(elem in vertex_ids for elem in t) for t in edge_vertex_id_pairs) == False:
        raise IDNotFoundError("Vertex id not found in edge array")
    

def check_length_enabled(edge_enabled, edge_ids): #checks if the number of enabled/disabled edges is the same as the number of edge ids
    if len(edge_enabled) != len(edge_ids):
        raise InputLengthDoesNotMatchError("Number of enabled and disabled edges does not match number of total edges")


def check_found_source(source_vertex_id, vertex_ids): #checks if the source vertex id is valid
    if source_vertex_id not in vertex_ids:
        raise IDNotFoundError("Source vertex id not found")


def check_connect(vertex_ids, edge_enabled, edge_vertex_id_pairs): #checks if graph is fully connected
    size = len(vertex_ids)
    sparseMatrix = [[0 for i in range(size)] for j in range(size)]
    for i in range(size):
        for j in range(size):
            if ((vertex_ids[i], vertex_ids[j]) in edge_vertex_id_pairs) and sparseMatrix[i][j] == 0:
                if edge_enabled[edge_vertex_id_pairs.index((vertex_ids[i], vertex_ids[j]))]:
                    sparseMatrix[i][j] = vertex_ids[j]
                    sparseMatrix[j][i] = vertex_ids[i]
            elif ((vertex_ids[j], vertex_ids[i]) in edge_vertex_id_pairs) and sparseMatrix[i][j] == 0:
                if edge_enabled[edge_vertex_id_pairs.index((vertex_ids[j], vertex_ids[i]))]:
                    sparseMatrix[i][j] = vertex_ids[j]
                    sparseMatrix[j][i] = vertex_ids[i]
    graph = csr_array(sparseMatrix)
    components = connected_components(graph)
    if components[0] > 1:
        raise GraphNotFullyConnectedError("Graph contains more than 1 component")


def check_cycle(edge_enabled, edge_vertex_id_pairs): #checks if graph contains cycles
    G = nx.Graph()
    for (u, v), enabled in zip(edge_vertex_id_pairs, edge_enabled):
        if enabled:
            G.add_edge(u, v)

    has_cycle = nx.is_forest(G)  # A forest is a graph with no undirected cycles
    if has_cycle == False:
        raise GraphCycleError("Graph contains a cycle")


def check_found_edges(disabled_edge_id, all_edges): #checks if the edge to be disabled has a valid edge id
    if disabled_edge_id not in all_edges:
        raise IDNotFoundError("Disabled edge id not found in edge array")
    

def check_disabled(disabled_edge_id, edge_ids, edge_enabled): #checks if the edge to be disabled is already disabled
    if edge_enabled[edge_ids.index(disabled_edge_id)] == False:
        raise EdgeAlreadyDisabledError("Edge is already disabled")

With all of the necessary needed external functions declared, the graph processing class can now also be created

In [15]:
class GraphProcessor(nx.Graph):
    def __init__(
        self,
        vertex_ids: List[int],
        edge_ids: List[int],
        edge_vertex_id_pairs: List[Tuple[int, int]],
        edge_enabled: List[bool],
        source_vertex_id: int,
    ) -> None:
        super().__init__()
        check_unique(vertex_ids, edge_ids)
        check_length_pairs(edge_vertex_id_pairs, edge_ids)
        check_found_pairs(edge_vertex_id_pairs, vertex_ids)
        check_length_enabled(edge_enabled, edge_ids)
        check_found_source(source_vertex_id, vertex_ids)
        check_connect(vertex_ids, edge_enabled, edge_vertex_id_pairs)
        check_cycle(edge_enabled, edge_vertex_id_pairs)

        self.source_vertex_id = source_vertex_id
        self.vertex_ids = vertex_ids
        self.edge_enabled = edge_enabled
        self.edge_ids = edge_ids
        self.edge_vertex_id_pairs = edge_vertex_id_pairs
        self.add_nodes_from(vertex_ids)
        for i, (u, v) in enumerate(edge_vertex_id_pairs):
            self.add_edge(u, v, id=edge_ids[i], enabled=edge_enabled[i])

        self.enabled_subgraph = nx.Graph()
        for (u, v), enabled in zip(edge_vertex_id_pairs, edge_enabled):
            if enabled:
                self.enabled_subgraph.add_edge(u, v)
        # DFS tree & parent map from source
        self.dfs_tree = nx.dfs_tree(self.enabled_subgraph, self.source_vertex_id)
        self.parent_map = {child: parent for parent, child in nx.dfs_edges(self.dfs_tree, source=self.source_vertex_id)}

    def find_downstream_vertices(self, edge_id: int) -> List[int]:
        if edge_id not in self.edge_ids:
            raise IDNotFoundError("Edge ID not found.")

        edge_index = self.edge_ids.index(edge_id)
        if not self.edge_enabled[edge_index]:
            return []

        u, v = self.edge_vertex_id_pairs[edge_index]

        # Ensure both u and v are reachable
        if u not in self.dfs_tree or v not in self.dfs_tree:
            return []

        # Determine downstream vertex (child in DFS tree)
        if self.parent_map.get(v) == u:
            downstream_root = v
        elif self.parent_map.get(u) == v:
            downstream_root = u
        else:
            # If neither is parent of the other, one of them is ancestor; pick the child
            # or fallback to whichever is deeper
            depth = nx.single_source_shortest_path_length(self.dfs_tree, self.source_vertex_id)
            downstream_root = v if depth.get(v, 0) > depth.get(u, 0) else u

        descendants = list(nx.descendants(self.dfs_tree, downstream_root))
        return [downstream_root] + descendants

    def find_alternative_edges(self, disabled_edge_id: int) -> List[int]:
        ans = []
        check_found_edges(disabled_edge_id, self.edge_ids)
        check_disabled(disabled_edge_id, self.edge_ids, self.edge_enabled)
        H = nx.Graph()
        for i, (u, v) in enumerate(self.edges()):
            if self.edge_enabled[i] == True and self.edge_ids[i] != disabled_edge_id:
                H.add_edge(u, v)
        for i, (u, v) in enumerate(self.edges):
            if self.edge_enabled[i] == False and self.edge_ids[i] != disabled_edge_id:
                H.add_edge(u, v)
                if nx.number_connected_components(H) == 1 and nx.is_forest(H):
                    ans.append(self.edge_ids[i])
                H.remove_edge(u, v)
        return ans

With the class implemented its functionalities can now be tested.

In [22]:
vertex_ids1 = [0, 2, 4, 6, 10]
edge_ids1 = [1, 3, 5, 7, 9, 8]
edge_vertex_id_pairs1 = [(0, 2), (0, 4), (0, 6), (2, 4), (2, 10), (4, 6)]
edge_enabled1 = [True, True, True, False, True, False]
source_vertex_id1 = 0

"""
vertex_0 (source) --edge_1(enabled)-- vertex_2 --edge_9(enabled)-- vertex_10
|                               |
|                           edge_7(disabled)
|                               |
-----------edge_3(enabled)-- vertex_4
|                               |
|                           edge_8(disabled)
|                               |
-----------edge_5(enabled)-- vertex_6
"""

test_graph1=GraphProcessor(vertex_ids1, edge_ids1, edge_vertex_id_pairs1, edge_enabled1, source_vertex_id1)
edge_to_disable=3
print(f"The list of alternative edges when disabling edge {edge_to_disable} is: {test_graph1.find_alternative_edges(edge_to_disable)}")


The list of alternative edges when disabling edge 3 is: [7, 8]


In [23]:
vertex_ids2 = [0, 2, 4, 6, 8, 10, 12]
edge_ids2 = [1, 3, 5, 7, 9, 11]
edge_vertex_id_pairs2 = [(0, 2), (2, 4), (2, 6), (4, 8), (8, 10), (6, 12)]
edge_enabled2 = [True, True, True, True, True, True]
source_vertex_id2 = 0

"""
    vertex_0 (source) --edge_1-- vertex_2 --edge_3-- vertex_4--edge 7--vertex 8 --edge 9--vertex 10
                                    |
                                  edge 5
                                    |
                                 vertex 6  --edge 11 --vertex 12
"""

test_graph2=GraphProcessor(vertex_ids2, edge_ids2, edge_vertex_id_pairs2, edge_enabled2, source_vertex_id2)
edge_down=3
print(f"The list of downstream vertices for edge {edge_down} is {test_graph2.find_downstream_vertices(edge_down)}")

The list of downstream vertices for edge 3 is [4, 8, 10]


# **Assignment 2 - power grid model**

# **Assignment 3 - developing a power system simulation package**